In [ ]:
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
tee = KeyedVectors.load_word2vec_format(datapath("/home/cremaschi/fede/teeentity:s100:w5type:s100:w5"), binary=False)

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
replacebad_chars = ["-", "'", "(", ")", "·", "1", ",", "ʻ", "–", "ø", "×", "/", "2", "’", "æ", "ł", "&", "ı", "5"]
replacebad_chars = ["-", "'", "(", ")", "·", "–", "ø", "×", "/", "2"]

def replace_chars(string):
    new_l = string
    for char in replacebad_chars:
        new_l = new_l.replace(char, "")
    return new_l

In [ ]:
vocab = set([k for k in tee.wv.vocab])
import tensorflow as tf

print(tf.test.gpu_device_name())

In [ ]:
import importlib
import tensorflow #import the module here, so that it can be reloaded.
importlib.reload(tensorflow)
importlib.reload(os)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Dataset 1

In [ ]:
training = []
testing = []

with open("setting_one_training", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        training.append(((a),b, c))
        
with open("setting_one_testing", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        testing.append(((a),b, c))
        
        
entities_to_train_on = set(map(lambda x : x[0], training))

# Dataset 2

In [ ]:
training = []
testing = []

with open("setting_two_training_big", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        training.append(((a),b, c))
        
with open("setting_two_testing_big", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        testing.append(((a),b, c))
        
        
entities_to_train_on = set(map(lambda x : x[0], training))

# Dataset 3

In [ ]:
training = []
testing = []

with open("setting_three_training", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        training.append(((a),b, c))
        
with open("setting_three_testing", "r") as file_tp:
    for line in file_tp:
        a, b, c = line.replace("\n", "").split("\t")
        testing.append(((a),b, c))
        
        
entities_to_train_on = set(map(lambda x : x[0], training))#.union(set(map(lambda x : x[0], testing)))

In [ ]:
classes = list(["Animal", "Fungus", "Mammal", "Plant", "Bacteria", "Eukaryote", "Species"])
def analyze_dataset(training, testing):
    len_training = len(training)
    len_testing = len(testing)
    for a in classes:
        a_in_training = list(filter(lambda x : x[1] == a, training))
        a_in_testing = list(filter(lambda x : x[1] == a, testing))
        
        pos_in_training = float(len(list(filter(lambda x : x[1] == a and int(x[2]) == 1, training))))
        pos_in_testing = float(len(list(filter(lambda x : x[1] == a and int(x[2]) == 1, testing))))
        
        print(a, "{0:.2f}".format((pos_in_training/len(a_in_training)))
                                  , "{0:.2f}".format(pos_in_testing/len(a_in_testing)))
    print(len_training, len_testing)

In [ ]:
analyze_dataset(training,testing)

# LTN

In [ ]:
ltnw._reset()


In [ ]:
import logging
logger = logging.getLogger()
logger.basicConfig = logging.basicConfig(level=logging.DEBUG)
import logictensornetworks_wrapper as ltnw
import tensorflow as tf
import numpy as np
import logictensornetworks as ltn
import itertools
import argparse

embedding_size = 100

ltn.LAYERS = 20
ltn.BIAS_factor = -1e-5
#ltn.set_universal_aggreg("mean")
ltn.set_tnorm("luk")
ltn.set_universal_aggreg("hmean")

ltnw.predicate("Plant", embedding_size)
ltnw.predicate("Eukaryote",embedding_size)
ltnw.predicate("Species",embedding_size)
ltnw.predicate("Mammal",embedding_size)
ltnw.predicate("Animal",embedding_size)
ltnw.predicate("Fungus",embedding_size)
ltnw.predicate("Bacteria",embedding_size)

replacebad_chars = ["-", "'", "(", ")", "·", "–", "ø", "×", "/", "2"]

for l in (entities_to_train_on):
    new_l = l
    for char in replacebad_chars:
        new_l = new_l.replace(char, "")
    #ltnw.constant(new_l, min_value=[0.] * embedding_size, max_value=[1.] * embedding_size)
    ltnw.constant(new_l, tee[l][100:])

for index, data in enumerate(training):
    new_l = data[0]
    c = data[1]
    #print(data[0])
    for char in replacebad_chars:
        new_l = new_l.replace(char, "")
    
    string = c  +"(" + new_l + ")"
    if data[2] == '0':
        string = "~" + string
    #logging.info(string)
    m = ltnw.axiom(string)
    if index%1000==0:
        print(index)
        


In [ ]:
tensor_data = np.array(list(map(lambda x : x[100:], tee[entities_to_train_on])))

ltnw.variable("?a",tensor_data);
#ltnw.variable("?a",tf.concat(list(ltnw.CONSTANTS.values()),axis=0))
ltnw.axiom("forall ?a: Plant(?a) -> Eukaryote(?a)")
ltnw.axiom("forall ?a: Animal(?a) -> Eukaryote(?a)")
ltnw.axiom("forall ?a: Fungus(?a) -> Eukaryote(?a)")
ltnw.axiom("forall ?a: Bacteria(?a) -> Species(?a)")
ltnw.axiom("forall ?a: Mammal(?a) -> Animal(?a)")
ltnw.axiom("forall ?a: Eukaryote(?a) -> Species(?a)")

ltnw.axiom("forall ?a: Plant(?a) -> ~Mammal(?a)")
ltnw.axiom("forall ?a: Plant(?a) -> ~Animal(?a)")
ltnw.axiom("forall ?a: Plant(?a) -> ~Fungus(?a)")
ltnw.axiom("forall ?a: Plant(?a) -> ~Bacteria(?a)")

ltnw.axiom("forall ?a: Mammal(?a) -> ~Plant(?a)")
ltnw.axiom("forall ?a: Mammal(?a) -> ~Fungus(?a)")
ltnw.axiom("forall ?a: Mammal(?a) -> ~Bacteria(?a)")

ltnw.axiom("forall ?a: Bacteria(?a) -> ~Mammal(?a)")
ltnw.axiom("forall ?a: Bacteria(?a) -> ~Eukaryote(?a)")
ltnw.axiom("forall ?a: Bacteria(?a) -> ~Animal(?a)")
ltnw.axiom("forall ?a: Bacteria(?a) -> ~Plant(?a)")
ltnw.axiom("forall ?a: Bacteria(?a) -> ~Fungus(?a)")

ltnw.axiom("forall ?a: Fungus(?a) -> ~Mammal(?a)")
ltnw.axiom("forall ?a: Fungus(?a) -> ~Animal(?a)")
ltnw.axiom("forall ?a: Fungus(?a) -> ~Plant(?a)")
ltnw.axiom("forall ?a: Fungus(?a) -> ~Bacteria(?a)")

In [ ]:
ltnw.initialize_knowledgebase(
    optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01,decay=0.9),
    formula_aggregator=lambda *x: 1./tf.reduce_mean(1./tf.concat(x,axis=0)))

# Train the KB
sat_level = ltnw.train(max_epochs=10000, track_sat_levels=10, sat_level_epsilon=.995)

## Test LTN

In [ ]:
testing_n = list(map(lambda k : k[1] + "(" + replace_chars(k[0]) + ")", testing))
testing_costant = set(map(lambda x : x[0], testing))
results = ltnw.ask_m(testing_n)
conca = list(zip(testing, res))

In [ ]:
for l in (testing_costant):
    new_l = l
    for char in replacebad_chars:
        new_l = new_l.replace(char, "")
    #ltnw.constant(new_l, min_value=[0.] * embedding_size, max_value=[1.] * embedding_size)
    ltnw.constant(new_l, tee[l][100:])

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
classes = set(["Plant", "Eukaryote", "Species", "Fungus", "Bacteria", "Animal", "Mammal"])

for a in classes:
    filtering = list(map(lambda x: (int(x[0][2]), round(x[1])), filter(lambda x : x[0][1] == a, conca)))
    filtering = [list(t) for t in zip(*filtering)]
    print(a, f1_score(filtering[0], filtering[1]))
filtering = list(map(lambda x: (int(x[0][2]), round(x[1])), filter(lambda x : x[0][1] != "asfa", conca)))
filtering = [list(t) for t in zip(*filtering)]
print("General", f1_score(filtering[0], filtering[1]))
print("Precision", precision_score(filtering[0], filtering[1]))
print("Recall", recall_score(filtering[0], filtering[1]))

# DNN

In [ ]:
import numpy as np

word2int = {}
int2word = {}
classes = set(["Plant", "Eukaryote", "Species", "Fungus", "Bacteria", "Animal", "Mammal"])

def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

vocab_size = len(classes)  # gives the total number of unique words

for i, word in enumerate(classes):
    word2int[word] = i
    int2word[i] = word
from keras import backend as K
import tensorflow as tf
def as_keras_metric(method):
    import functools
    
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper


In [ ]:
# convert them to numpy arrays
train_clazz = []  # input word
entity_one_train = []  # input word
y_train = []  # output word

for a,b,c in training:
    clazz = to_one_hot(word2int[b], vocab_size)
    second = tee[a][100:]
    
    entity_one_train.append(second)
    train_clazz.append(clazz)

    y_train.append([c])



train_clazz = np.asarray(train_clazz)
entity_one_train = np.asarray(entity_one_train)
y_train = np.asarray(y_train)

In [ ]:
# convert them to numpy arrays
test_clazz = []  # input word
entity_one_test = []  # input word

y_test = []  # output word

for a,b,c in testing:
    clazz = to_one_hot(word2int[b], vocab_size)
    second = tee[a][100:]
    
    entity_one_test.append(second)
    test_clazz.append(clazz)

    y_test.append([c])



test_clazz = np.asarray(test_clazz)
entity_one_test = np.asarray(entity_one_test)
y_test = np.asarray(y_test)

In [ ]:
from keras.models import Model
from keras import layers, regularizers
from keras.layers import Input

In [ ]:
import keras_metrics
from keras.callbacks import EarlyStopping
entity_vocabulary_size = len(entities_to_train_on)

prop_vocabulary_size = 7

entity_one_input = Input(shape=(100,))
entity_one_first_step = layers.Dense(20, activation="relu")(entity_one_input)

entity_two_input = Input(shape=(prop_vocabulary_size,))
entity_two_first_step = layers.Dense(20, activation="relu")(entity_two_input)


concatenated = layers.concatenate([entity_one_first_step, entity_two_first_step], axis=-1)

belta = layers.Dense(20, activation="relu")(concatenated)
dropout = layers.Dropout(0.2)(belta)
answer = layers.Dense(1, activation='sigmoid')(dropout)

model = Model([entity_one_input, entity_two_input], answer)
precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[ 
                                                                        keras_metrics.precision(), 
                                                                        keras_metrics.recall(),
                                                                        keras_metrics.f1_score()])




In [ ]:
model.fit([entity_one_train,train_clazz], y_train, epochs=3000, batch_size=500,  callbacks =[es], validation_split=0.2)


## Test DNN

In [ ]:
dada = (model.evaluate([entity_one_test,test_clazz], y_test))

print(dada)

In [ ]:
res_dnn = model.predict([entity_one_test,test_clazz])
res_dnn_rounded = list(map(lambda x : round(x[0]), res_dnn))
zipped_test_results_collection = list(zip(testing, res_dnn_rounded))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
classes = set(["Plant", "Eukaryote", "Species", "Fungus", "Bacteria", "Animal", "Mammal"])

for a in classes:
    filtering = list(map(lambda x: (int(x[0][2]), round(x[1])), filter(lambda x : x[0][1] == a, zipped_test_results_collection)))
    filtering = [list(t) for t in zip(*filtering)]
    print(a, f1_score(filtering[0], filtering[1]))
filtering = list(map(lambda x: (int(x[0][2]), round(x[1])), filter(lambda x : x[0][1] != "asfa", zipped_test_results_collection)))
filtering = [list(t) for t in zip(*filtering)]
print("General", f1_score(filtering[0], filtering[1]))
print("Precision", precision_score(filtering[0], filtering[1]))
print("Recall", recall_score(filtering[0], filtering[1]))